In [25]:
import subprocess, sys
import opc
from PIL import Image
from io import BytesIO
import os

In [26]:
# Tensorflow variables
IMAGE_SIZE = 224
ARCHITECTURE = "mobilenet_0.50_" + str(IMAGE_SIZE)

# Storage variables
CLOUD_FOLDER = "flower_photos"
LOCAL_FOLDER = "tf_files/"
extensionsToCheck = ('.jpg', '.png')

In [27]:
def get_images():
    print("Listing images from cloud folder...")
    _, cloud_files = opc_storage.show_container_details_and_list_objects(CLOUD_FOLDER)
    # Filtering images
    cloud_images = [ cloud_file for cloud_file in cloud_files if cloud_file.endswith(extensionsToCheck) ]
    
    print("Listing images from local folder...")
    local_images = []
    for dirpath, _, filenames in os.walk(LOCAL_FOLDER + CLOUD_FOLDER):
        for filename in filenames:
            #local_images.append(dirpath + "/" + filename)
            local_images.append(dirpath.replace(LOCAL_FOLDER, "").replace("\\", "/") + "/" + filename)

    # Verify Removed and Added images in the cloud
    old_local_images = list(set(local_images) - set(cloud_images))
    new_cloud_images = list(set(cloud_images) - set(local_images))
    
    # Remove old local images
    if len(old_local_images) > 0:
        count = 0
        print("Removing", len(old_local_images), "file(s):")
        for old_local_image in old_local_images:
            count += 1
            print(count, "-", LOCAL_FOLDER + old_local_image, "...")
            os.remove(LOCAL_FOLDER + old_local_image)
            
    # Download new cloud images
    if len(new_cloud_images) > 0:
        count = 0
        print("Adding", len(new_cloud_images), "file(s):")
        for new_cloud_image in new_cloud_images:
            directory = os.path.dirname(LOCAL_FOLDER + new_cloud_image)
            if not os.path.exists(directory):
                os.makedirs(directory)
                print("Folder", directory, "created!")                

            count += 1
            print(count, "-", LOCAL_FOLDER + new_cloud_image, "...")
            _, file = opc_storage.get_object_content_and_metadata(CLOUD_FOLDER, new_cloud_image)
            im = Image.open(BytesIO(file))
            im.save(LOCAL_FOLDER + new_cloud_image)
            
    print("Done!")

In [28]:
opc_storage = opc.Storage("waslley.souza@oracle.com", "CDfv@er55", "gse00014123")
get_images()

Listing images from cloud folder...
Listing images from local folder...
Done!


In [29]:
def execute(command):
    process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)

    # Poll process for new output until finished
    while True:
        nextline = process.stdout.readline()
        if len(nextline) == 0 and process.poll() is not None:
            break
        sys.stdout.write(nextline)
        sys.stdout.flush()

    output = process.communicate()[0]
    exitCode = process.returncode

    if (exitCode == 0):
        return output
    else:
        raise ProcessException(command, exitCode, output)

In [32]:
cmd = '''python scripts/retrain.py \
  --bottleneck_dir=tf_files/bottlenecks \
  --how_many_training_steps=10 \
  --model_dir=tf_files/models/ \
  --summaries_dir=tf_files/training_summaries/{ARCHITECTURE} \
  --output_graph=tf_files/retrained_graph.pb \
  --output_labels=tf_files/retrained_labels.txt \
  --architecture={ARCHITECTURE} \
  --image_dir=tf_files/flower_photos'''.format(ARCHITECTURE=ARCHITECTURE)

print(cmd)
# execute(cmd)

python scripts/retrain.py   --bottleneck_dir=tf_files/bottlenecks   --how_many_training_steps=10   --model_dir=tf_files/models/   --summaries_dir=tf_files/training_summaries/mobilenet_0.50_224   --output_graph=tf_files/retrained_graph.pb   --output_labels=tf_files/retrained_labels.txt   --architecture=mobilenet_0.50_224   --image_dir=tf_files/flower_photos


In [31]:
cmd = '''python scripts/label_image.py \
  --graph=tf_files/retrained_graph.pb  \
  --image=tf_files/flower_photos/daisy/21652746_cc379e0eea_m.jpg'''

# print(cmd)
execute(cmd)

D:\oracle\apps\anaconda3\envs\py36\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters

Evaluation time (1-image): 0.637s

roses 0.79813194
sunflowers 0.19288608
daisy 0.0060700956
dandelion 0.002060491
tulips 0.0008513967
2018-05-30 19:51:11.387639: I T:\src\github\tensorflow\tensorflow\core\platform\cpu_feature_guard.cc:140] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2


b''